# Use Case 1: "Give Me Routes Like This"

**Purpose:** Given a set of route features, find the most similar routes in our dataset.

**Model:** KNN with k=5, cosine metric (93.9% improvement over baseline)

**Use Cases:**
- Cyclist wants routes similar to one they enjoyed
- Find alternatives in different locations with same characteristics
- Discover routes with similar difficulty/distance/terrain

## 1. Setup and Load Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors

# Load data
df = pd.read_csv("UK_Engineered_Data.csv")
pd.set_option('display.max_columns', None)
print(f"Loaded {len(df)} routes")

Loaded 7717 routes


## 2. Prepare Features and Scaler

In [2]:
# Create feature matrix
X = df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'id', 'name'], axis=1)

# Apply same scaling as training
scaler = ColumnTransformer(transformers=[
    ('standard', StandardScaler(), ['distance_m', 'duration_s', 'ascent_m', 'descent_m', 'Turn_Density', 'Average_Speed', 'steps', 'turns']),
    ('minmax', MinMaxScaler(), ['Asphalt', 'Unknown', 'Paved', 'Compacted Gravel', 'Wood', 'Gravel', 'Paving Stones', 'Ground', 'Concrete', 'Grass', 'Metal', 'Unpaved', 'Dirt', 'Grass Paver', 'Sand', 'Road', 'Cycleway', 'State Road', 'Track', 'Street', 'Path', 'Footway', 'Unknown.1', 'Steps', 'Construction', 'Ferry', 'uphill_very_steep (7% to 10%)', 'uphill_moderate (3% to 5%)', 'uphill_gentle (0% to 3%)', 'flat (0%)', 'downhill_gentle (-5% to 0%)', 'uphill_steep (5% to 7%)', 'uphill_extreme (>10%)', 'downhill_extreme (<-15%)', 'downhill_moderate (-7% to -5%)', 'downhill_steep (-10% to -7%)', 'downhill_very_steep (-15% to -10%)']),
], remainder='passthrough')

X_scaled = scaler.fit_transform(X)
print(f"Features prepared: {X.shape[1]} features")

Features prepared: 45 features


## 3. Train Optimized KNN Model

Using optimal parameters from grid search: k=5, cosine metric

In [3]:
# Train with optimal parameters
knn_optimal = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_optimal.fit(X_scaled)

print("✅ Optimized KNN model trained!")
print(f"   k = {knn_optimal.n_neighbors}")
print(f"   metric = '{knn_optimal.metric}'")
print(f"   Training samples = {X_scaled.shape[0]}")

✅ Optimized KNN model trained!
   k = 5
   metric = 'cosine'
   Training samples = 7717


---
# PRODUCTION RECOMMENDATION FUNCTION

## 4. Define Recommendation Function

In [4]:
def recommend_similar_routes(input_features, n_recommendations=5, show_details=True):
    """
    Find routes similar to given features

    Parameters:
    - input_features: Dict, Series, or DataFrame with route features (must match X columns)
    - n_recommendations: Number of similar routes to return (default 5)
    - show_details: If True, prints detailed comparison (default True)

    Returns:
    - DataFrame with recommended routes and similarity scores
    """
    # Convert input to DataFrame
    if isinstance(input_features, dict):
        input_df = pd.DataFrame([input_features])
    elif isinstance(input_features, pd.Series):
        input_df = input_features.to_frame().T
    else:
        input_df = input_features.copy()

    # Validate columns
    if not all(col in input_df.columns for col in X.columns):
        missing = [col for col in X.columns if col not in input_df.columns]
        print(f"❌ Error: Missing features: {missing}")
        return None

    # Ensure column order matches
    input_df = input_df[X.columns]

    # Print input summary
    print("=" * 80)
    print("INPUT ROUTE FEATURES")
    print("=" * 80)
    print(f"Distance: {input_df['distance_m'].values[0]:.1f}m")
    print(f"Ascent: {input_df['ascent_m'].values[0]:.1f}m")
    print(f"Duration: {input_df['duration_s'].values[0]:.1f}s")
    print(f"Avg Speed: {input_df['Average_Speed'].values[0]:.2f}")
    print(f"Turn Density: {input_df['Turn_Density'].values[0]:.2f}")
    print()

    # Scale features
    input_scaled = scaler.transform(input_df)

    # Find nearest neighbors
    distances, indices = knn_optimal.kneighbors(input_scaled, n_neighbors=n_recommendations)

    # Build results
    recommendations = []
    for rank, (dist, idx) in enumerate(zip(distances[0], indices[0]), 1):
        rec_route = df.iloc[idx]

        recommendations.append({
            'rank': rank,
            'route_id': rec_route['id'],
            'route_name': rec_route['name'],
            'distance_m': rec_route['distance_m'],
            'ascent_m': rec_route['ascent_m'],
            'duration_s': rec_route['duration_s'],
            'avg_speed': rec_route['Average_Speed'],
            'turn_density': rec_route['Turn_Density'],
            'similarity_score': dist
        })

    results_df = pd.DataFrame(recommendations)

    # Print results
    print("=" * 80)
    print(f"TOP {n_recommendations} SIMILAR ROUTES")
    print("=" * 80)
    print()

    for i, row in results_df.iterrows():
        dist_diff = row['distance_m'] - input_df['distance_m'].values[0]
        ascent_diff = row['ascent_m'] - input_df['ascent_m'].values[0]

        print(f"{row['rank']}. {row['route_name'][:60]}")
        print(f"   Route ID: {row['route_id']}")
        print(f"   Similarity: {row['similarity_score']:.4f} (lower = more similar)")
        print(f"   Distance: {row['distance_m']:.1f}m ({dist_diff:+.1f}m)")
        print(f"   Ascent: {row['ascent_m']:.1f}m ({ascent_diff:+.1f}m)")
        print(f"   Duration: {row['duration_s']:.1f}s")
        print(f"   Avg Speed: {row['avg_speed']:.2f}")
        print()

    # Detailed comparison
    if show_details:
        print("=" * 80)
        print("FEATURE COMPARISON")
        print("=" * 80)
        print(f"{'Feature':<25} {'Input':<15} {'Avg Recommended':<20} {'Difference':<15}")
        print("-" * 80)

        # Map between input column names and results_df column names
        feature_mapping = [
            ('distance_m', 'distance_m'),
            ('ascent_m', 'ascent_m'),
            ('duration_s', 'duration_s'),
            ('Average_Speed', 'avg_speed'),      # results_df uses lowercase
            ('Turn_Density', 'turn_density')      # results_df uses lowercase
        ]

        for input_col, results_col in feature_mapping:
            input_val = input_df[input_col].values[0]
            avg_rec_val = results_df[results_col].mean()
            diff = avg_rec_val - input_val
            print(f"{input_col:<25} {input_val:<15.2f} {avg_rec_val:<20.2f} {diff:+.2f}")

    print("\n" + "=" * 80)

    return results_df

print("✅ Recommendation function ready!")

✅ Recommendation function ready!


## 5. Helper Function: Get Features from Route ID

In [5]:
def get_route_features(route_id):
    """
    Extract features from an existing route by ID
    Useful for testing with known routes

    Parameters:
    - route_id: The route ID to look up

    Returns:
    - Series with route features
    """
    if route_id not in df['id'].values:
        print(f"❌ Error: Route ID {route_id} not found in dataset")
        return None

    route = df[df['id'] == route_id].iloc[0]
    features = route[X.columns]

    print(f"✅ Extracted features from: '{route['name']}' (ID: {route_id})")
    print(f"   Distance: {route['distance_m']:.1f}m, Ascent: {route['ascent_m']:.1f}m")
    print()

    return features

print("✅ Helper function ready!")

✅ Helper function ready!


---
# EXAMPLES

## Example 1: Use Existing Route Features

In [6]:
# Pick a route from the dataset
example_route_id = df.iloc[100]['id']

# Get its features
features = get_route_features(example_route_id)

# Find similar routes
recommendations = recommend_similar_routes(features, n_recommendations=5)

✅ Extracted features from: 'Salle Cycle Loop' (ID: 18923017)
   Distance: 2031.1m, Ascent: 6.7m

INPUT ROUTE FEATURES
Distance: 2031.1m
Ascent: 6.7m
Duration: 406.2s
Avg Speed: 5.00
Turn Density: 0.00

TOP 5 SIMILAR ROUTES

1. Salle Cycle Loop
   Route ID: 18923017
   Similarity: 0.0000 (lower = more similar)
   Distance: 2031.1m (+0.0m)
   Ascent: 6.7m (+0.0m)
   Duration: 406.2s
   Avg Speed: 5.00

2. Unnamed route
   Route ID: 15590082
   Similarity: 0.0001 (lower = more similar)
   Distance: 1797.5m (-233.6m)
   Ascent: 6.0m (-0.7m)
   Duration: 359.5s
   Avg Speed: 5.00

3. Unnamed route
   Route ID: 15588106
   Similarity: 0.0001 (lower = more similar)
   Distance: 1634.9m (-396.2m)
   Ascent: 4.3m (-2.4m)
   Duration: 327.0s
   Avg Speed: 5.00

4. Unnamed route
   Route ID: 15588105
   Similarity: 0.0002 (lower = more similar)
   Distance: 2055.2m (+24.1m)
   Ascent: 15.4m (+8.7m)
   Duration: 411.0s
   Avg Speed: 5.00

5. Hampden Route
   Route ID: 12890644
   Similarity: 0.000

## Example 2: Manual Feature Input (Short, Flat Route)

In [7]:
# User wants: short, flat, paved route
custom_features = {
    'distance_m': 3000.0,        # 3km
    'duration_s': 600.0,         # 10 minutes
    'ascent_m': 100.0,            # Not Very flat
    'descent_m': 100.0,
    'steps': 3,
    'turns': 5,
    'Asphalt': 90.0,             # 90% paved
    'Unknown': 0.0,
    'Paved': 10.0,
    'Compacted Gravel': 0.0,
    'Wood': 0.0,
    'Gravel': 0.0,
    'Paving Stones': 0.0,
    'Ground': 0.0,
    'Concrete': 0.0,
    'Grass': 0.0,
    'Metal': 0.0,
    'Unpaved': 0.0,
    'Dirt': 0.0,
    'Grass Paver': 0.0,
    'Sand': 0.0,
    'Road': 100.0,
    'Cycleway': 0.0,
    'State Road': 0.0,
    'Track': 0.0,
    'Street': 0.0,
    'Path': 0.0,
    'Footway': 0.0,
    'Unknown.1': 0.0,
    'Steps': 0.0,
    'Construction': 0.0,
    'Ferry': 0.0,
    'uphill_very_steep (7% to 10%)': 0.0,
    'uphill_moderate (3% to 5%)': 0.0,
    'uphill_gentle (0% to 3%)': 10.0,
    'flat (0%)': 90.0,           # Mostly flat
    'downhill_gentle (-5% to 0%)': 0.0,
    'uphill_steep (5% to 7%)': 0.0,
    'uphill_extreme (>10%)': 0.0,
    'downhill_extreme (<-15%)': 0.0,
    'downhill_moderate (-7% to -5%)': 0.0,
    'downhill_steep (-10% to -7%)': 0.0,
    'downhill_very_steep (-15% to -10%)': 0.0,
    'Average_Speed': 5.0,
    'Turn_Density': 1.67
}

print("\n" + "#" * 80)
print("# EXAMPLE 2: Short, Flat, Paved Route")
print("#" * 80)
print()

recommendations2 = recommend_similar_routes(custom_features, n_recommendations=5)


################################################################################
# EXAMPLE 2: Short, Flat, Paved Route
################################################################################

INPUT ROUTE FEATURES
Distance: 3000.0m
Ascent: 100.0m
Duration: 600.0s
Avg Speed: 5.00
Turn Density: 1.67

TOP 5 SIMILAR ROUTES

1. Unnamed route
   Route ID: 18053035
   Similarity: 0.0258 (lower = more similar)
   Distance: 7472.0m (+4472.0m)
   Ascent: 80.0m (-20.0m)
   Duration: 1494.4s
   Avg Speed: 5.00

2. Kingfisher Cycle Trail - Clones - Newbliss - Scotshouse (Rid
   Route ID: 1709339
   Similarity: 0.0301 (lower = more similar)
   Distance: 5262.4m (+2262.4m)
   Ascent: 73.2m (-26.8m)
   Duration: 1052.5s
   Avg Speed: 5.00

3. Unnamed route
   Route ID: 15567652
   Similarity: 0.0318 (lower = more similar)
   Distance: 7251.2m (+4251.2m)
   Ascent: 62.4m (-37.6m)
   Duration: 1450.2s
   Avg Speed: 5.00

4. Unnamed route
   Route ID: 19018904
   Similarity: 0.0327 (lower = more

---
## Summary

**Use Case 1 Function Complete!**

**How to use:**
```python
# Option 1: From existing route
features = get_route_features(route_id)
recommendations = recommend_similar_routes(features)

# Option 2: Custom features
custom = {'distance_m': 5000, 'ascent_m': 50, ...}
recommendations = recommend_similar_routes(custom)
```

**Returns:**
- DataFrame with route IDs, names, features, and similarity scores
- Prints detailed comparison

**Next:** Use Case 2 (custom modifications like "2x distance")

## Example Three

In [8]:
def extract_interpretable_features(row):
    """
    Converts a full engineered feature row into an interpretable, LLM-friendly
    feature dictionary (Option B: grouped/aggregated features).
    """

    # --- 1. Core numeric features ---
    features = {
        "distance_m": float(row["distance_m"]),
        "duration_s": float(row["duration_s"]),
        "ascent_m": float(row["ascent_m"]),
        "descent_m": float(row["descent_m"])
    }

    # --- 2. Surface Profile ---
    surface_map = {
        "paved_percent": ["Asphalt", "Paved", "Concrete"],
        "gravel_percent": ["Gravel", "Compacted Gravel"],
        "dirt_percent": ["Dirt", "Unpaved", "Ground"],
        "grass_percent": ["Grass", "Grass Paver"],
        "other_percent": ["Wood", "Metal", "Sand", "Paving Stones", "Unknown"]
    }

    surface_profile = {}
    for key, cols in surface_map.items():
        surface_profile[key] = float(sum(row[col] for col in cols))

    # --- 3. Hill Profile ---
    hill_map = {
        "flat_percent": ["flat (0%)"],
        "gentle_uphill_percent": ["uphill_gentle (0% to 3%)"],
        "moderate_uphill_percent": ["uphill_moderate (3% to 5%)"],
        "steep_uphill_percent": ["uphill_steep (5% to 7%)", "uphill_very_steep (7% to 10%)", "uphill_extreme (>10%)"],
        "gentle_downhill_percent": ["downhill_gentle (-5% to 0%)"],
        "steep_downhill_percent": ["downhill_moderate (-7% to -5%)", "downhill_steep (-10% to -7%)", "downhill_very_steep (-15% to -10%)", "downhill_extreme (<-15%)"]
    }

    hill_profile = {}
    for key, cols in hill_map.items():
        hill_profile[key] = float(sum(row[col] for col in cols))

    # --- 4. Shape profile ---
    route_shape = {
        "turns": float(row["turns"]),
        "steps": float(row["steps"]),
        "turn_density": float(row["Turn_Density"]),
        "avg_speed": float(row["Average_Speed"])
    }

    return {
        "distance_m": features["distance_m"],
        "duration_s": features["duration_s"],
        "ascent_m": features["ascent_m"],
        "descent_m": features["descent_m"],
        "surface_profile": surface_profile,
        "hill_profile": hill_profile,
        "route_shape": route_shape
    }

print("✅ Interpretable feature extractor ready.")


✅ Interpretable feature extractor ready.


In [9]:
X

,distance_m,duration_s,ascent_m,descent_m,steps,turns,Asphalt,Unknown,Paved,Compacted Gravel,Wood,Gravel,Paving Stones,Ground,Concrete,Grass,Metal,Unpaved,Dirt,Grass Paver,Sand,Road,Cycleway,State Road,Track,Street,Path,Footway,Unknown.1,Steps,Construction,Ferry,uphill_very_steep (7% to 10%),uphill_moderate (3% to 5%),uphill_gentle (0% to 3%),flat (0%),downhill_gentle (-5% to 0%),uphill_steep (5% to 7%),uphill_extreme (>10%),downhill_extreme (<-15%),downhill_moderate (-7% to -5%),downhill_steep (-10% to -7%),downhill_very_steep (-15% to -10%),Average_Speed,Turn_Density
0,67.3,13.5,0.0,0.0,2,0,100.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.985185,0.000000
1,1617.8,323.6,88.9,1.9,2,0,0.00,100.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,24.14,13.79,62.07,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.999382,0.000000
2,149.6,29.9,2.0,0.0,2,0,100.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.003344,0.000000
3,88.7,17.7,3.0,0.0,2,0,100.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,100.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.011299,0.000000
4,910.0,203.6,0.6,10.6,2,0,18.18,81.82,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,81.82,18.18,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,100.0,0.0,0.0,0.0,0.0,0.0,0.0,4.469548,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7712,2447.3,515.9,19.2,18.2,13,11,92.08,0.99,0.00,0.00,0.0,0.0,6.93,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,25.74,56.44,10.89,0.00,3.96,0.0,2.97,0.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.743749,4.494749
7713,937.6,328.9,4.2,3.4,13,9,40.00,60.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,28.00,10.00,12.00,0.00,14.00,4.0,10.00,22.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.850715,9.598976
7714,1992.5,399.7,19.4,29.4,13,10,87.06,0.00,11.76,0.00,0.0,0.0,0.00,0.0,1.18,0.0,0.0,0.0,0.0,0.0,0.0,71.76,18.82,0.00,0.00,8.24,0.0,1.18,0.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.984989,5.018821
7715,3196.1,660.6,11.6,14.6,27,25,65.25,13.56,16.10,0.00,0.0,0.0,3.39,0.0,1.69,0.0,0.0,0.0,0.0,0.0,0.0,27.12,44.07,0.00,0.00,25.42,0.0,3.39,0.00,0.0,0.0,0.0,0.00,0.00,0.00,100.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.838177,7.822033


In [10]:
from langchain_core.prompts import PromptTemplate

In [11]:
route_edit_prompt = PromptTemplate(
    input_variables=["features", "instruction"],
    template="""
You are an assistant that modifies cycling route features based on a user's instruction.

You will receive:
1. An interpretable cycling route feature dictionary.
2. A natural-language instruction from the user.

Your job:
- Understand the user's intent.
- Create a JSON object describing how each feature should be changed.
- Only modify features that are relevant to the user's request.
- Do NOT produce any explanation or commentary — output JSON ONLY.

The JSON format must be:

{{
  "distance_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "duration_s": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "ascent_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},
  "descent_m": {{"operation": "<set/add/multiply/none>", "value": <number>}},

  "surface_profile": {{
      "paved_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "gravel_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "dirt_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "grass_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "other_percent": {{"operation": "<add/set/none>", "value": <number>}}
  }},

  "hill_profile": {{
      "flat_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "gentle_uphill_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "moderate_uphill_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "steep_uphill_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "gentle_downhill_percent": {{"operation": "<add/set/none>", "value": <number>}},
      "steep_downhill_percent": {{"operation": "<add/set/none>", "value": <number>}}
  }},

  "route_shape": {{
      "turns": {{"operation": "<add/set/none>", "value": <number>}},
      "steps": {{"operation": "<add/set/none>", "value": <number>}},
      "turn_density": {{"operation": "<add/set/none>", "value": <number>}},
      "avg_speed": {{"operation": "<add/set/none>", "value": <number>}}
  }}
}}

Rules:
- Use "none" if the user did NOT request a change.
- Use "multiply" for size-based instructions ("twice as long").
- Use "add" for relative changes ("a bit more gravel").
- Use "set" for absolute values ("make it 60 km long").
- Always return ALL fields, even if "none".

Here are the cycling route features:
{features}

User instruction:
"{instruction}"

Return ONLY the JSON object.
"""
)

print("✅ Cycling route prompt template loaded.")


✅ Cycling route prompt template loaded.


In [12]:
def apply_operation(original_value, op_dict):
    """
    Apply an operation (set/add/multiply/none) to a numerical feature.
    """
    operation = op_dict.get("operation", "none")
    value = op_dict.get("value", 0)

    if operation == "none":
        return original_value
    elif operation == "set":
        return float(value)
    elif operation == "add":
        return float(original_value) + float(value)
    elif operation == "multiply":
        return float(original_value) * float(value)
    else:
        return original_value  # fallback


def apply_transformation_plan(original_features, plan):
    """
    Applies the LLM-generated transformation plan to the interpretable cycling route features.
    Returns the updated feature dictionary.
    """

    updated = {}

    # 1. Core numeric features
    for key in ["distance_m", "duration_s", "ascent_m", "descent_m"]:
        updated[key] = apply_operation(original_features[key], plan[key])

    # 2. Surface profile
    updated_surface = {}
    for surf_key in original_features["surface_profile"].keys():
        updated_surface[surf_key] = apply_operation(
            original_features["surface_profile"][surf_key],
            plan["surface_profile"][surf_key]
        )

    # Normalize to sum to ~100
    total = sum(updated_surface.values())
    if total > 0:
        updated_surface = {k: (v / total) * 100 for k, v in updated_surface.items()}

    # 3. Hill profile
    updated_hill = {}
    for hill_key in original_features["hill_profile"].keys():
        updated_hill[hill_key] = apply_operation(
            original_features["hill_profile"][hill_key],
            plan["hill_profile"][hill_key]
        )

    # Normalize to sum to 100 again
    total_h = sum(updated_hill.values())
    if total_h > 0:
        updated_hill = {k: (v / total_h) * 100 for k, v in updated_hill.items()}

    # 4. Route shape fields
    updated_shape = {}
    for shape_key in original_features["route_shape"].keys():
        updated_shape[shape_key] = apply_operation(
            original_features["route_shape"][shape_key],
            plan["route_shape"][shape_key]
        )

    # Clamp invalid values
    updated_shape["turns"] = max(0, updated_shape["turns"])
    updated_shape["steps"] = max(0, updated_shape["steps"])

    updated["surface_profile"] = updated_surface
    updated["hill_profile"] = updated_hill
    updated["route_shape"] = updated_shape

    return updated

print("✅ Transformation plan application function ready.")


✅ Transformation plan application function ready.


In [13]:
from langchain_community.llms import HuggingFaceHub

In [14]:
from dotenv import load_dotenv
import os
load_dotenv()  # this reads your .env file

True

In [15]:
hf_token = os.getenv("HFT")
print("HF token loaded:", hf_token[:10] + "...")

HF token loaded: "hf_KXrada...


In [20]:
# Use HuggingFaceHub (already imported from langchain_community) instead of unavailable langchain_huggingface
llm = HuggingFaceHub(
    repo_id="google/gemma-2-9b-it",
    task="conversational",
    huggingfacehub_api_token=hf_token,
    model_kwargs={
        "temperature": 0.1,
        "max_new_tokens": 512
    }
)


In [21]:
import json as _json

# Patch InferenceClient to look like old API
def _post_patch(json=None, task=None):
    prompt = json["inputs"]
    params = json.get("parameters", {})
    res = llm.client.text_generation(prompt, **params)
    text = res if isinstance(res, str) else res.generated_text
    # match old .post return shape used by HuggingFaceHub
    return _json.dumps([{ "generated_text": text }]).encode()

llm.client.post = _post_patch
print("✅ Patched InferenceClient.post for HuggingFaceHub")


✅ Patched InferenceClient.post for HuggingFaceHub


In [22]:
import json

# Pick a route and make it interpretable
row = df.sample(1).iloc[0]
features = extract_interpretable_features(row)

# What you want the LLM to change
instruction = "Make it about 2x longer, mostly paved, fewer turns"

prompt = route_edit_prompt.format(
    features=json.dumps(features, indent=2),
    instruction=instruction
)

raw_plan = llm.invoke(prompt)
plan = json.loads(raw_plan)
updated = apply_transformation_plan(features, plan)
updated


ValueError: Model google/gemma-2-9b-it is not supported for task text-generation and provider nebius. Supported task: conversational.